In [2]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange,tqdm
import argparse
import xlrd
from datetime import datetime,timedelta
import dateparser
import re

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
def getloclist(wd,end,filter_out):
    loc_list = []
    EIRA_found = False
    for root, dirs, files in os.walk(wd):
        for file in files:
            if file.endswith(end) and not file.endswith(filter_out):
                loc = os.path.join(root, file)
                loc_list.append(loc) #get bim file location
#     if EIRA_found == True:""
#         print("EIRA detected!")
#     else:
#         print("Warning! No EIRA bim detected! Use first scanned cohort as ref!")
    return loc_list

In [5]:
def openfile(file,n=0,sep=','):
    if file.endswith((".csv",".CSV")):
        clinical = pd.read_csv(file,sep=sep,skiprows=n)
    elif file.endswith((".xlsx",".xls")):
        clinical = pd.read_excel(file,skiprows=n)
    elif file.endswith(".dta"):
        clinical = pd.read_stata(file)
    else:
#         raise ValueError("no clinical file detected!")
        clinical = pd.DataFrame()
    return clinical
def search(columns,patterns=["sjc","crp","sdai","cdai","mtx","Methotrexate"]):
    list_found = []
    for i in trange(len(columns)):
        for j in patterns:
            if j.lower() in columns[i].lower():
                print("Found pattern",j ,"in column num",i,columns[i])
                list_found.append(columns[i])
    return list_found
def search_folder(loc):
    list_found = []
    for root, dirs, files in os.walk(loc):
        for file in files:
            path = os.path.join(root, file)
            opened = openfile(path)
            print("openning file ",file)
            list_found.append(search(opened.columns))
    return list_found

In [6]:
def todatetime(a):
    b = dateparser.parse(a)
#     b = datetime.strptime(b,"%d-%b-%y")
    return b

In [7]:
def countRemission(fam):
    count_case = len(fam[fam[5]==2])
    count_control = len(fam[fam[5]==1])
    count_miss = len(fam[fam[5]==-9])+len(fam[fam[5]==0])
    plt.bar(["case",'control','missing'],[count_case,count_control,count_miss])
    return count_case,count_control,count_miss

In [8]:
# The primary outcome measure to model response trajectories was the disease activity score based on the 2-component DAS28, computed as 
# sqrt (SJC) + (0.6*ln[CRP+1]).
def TwocompDAS(sjc,crp):
    DAS = np.sqrt(sjc)+(0.6*np.log(crp+1))
    return DAS

In [9]:
def cDAI_S(cdai):
# sDAI <=3.3 case, >3.3 control
# cDAI <=2.8 case, >2.8 control
# ('1' = control, '2' = case)
    if cdai == -9:
        state = 0
    elif cdai <= 2.8:
        state = 2
    elif cdai > 2.8:
        state = 1
    return state

In [10]:
def sDAI_S(cdai):
# sDAI <=3.3 case, >3.3 control
# cDAI <=2.8 case, >2.8 control
# ('1' = control, '2' = case)
    if cdai == -9:
        state = 0
    elif cdai > 3.3:
        state = 1
    elif cdai <= 3.3:
        state = 2
    return state

## RAMS


In [16]:
RAMS_fam = pd.read_csv('./Pipelinerun_withPC/RAMS/mergedChr/RAMS_merged.fam',sep='\s',header=None)
RAMS_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,RAMS_1_RAMS_1,RAMS_1_RAMS_1,0,0,0,1
1,RAMS_10_RAMS_10,RAMS_10_RAMS_10,0,0,0,1
2,RAMS_100_RAMS_100,RAMS_100_RAMS_100,0,0,0,1
3,RAMS_102_RAMS_102,RAMS_102_RAMS_102,0,0,0,1
4,RAMS_103_RAMS_103,RAMS_103_RAMS_103,0,0,0,1
...,...,...,...,...,...,...
240,RAMS_89_RAMS_89,RAMS_89_RAMS_89,0,0,0,1
241,RAMS_91_RAMS_91,RAMS_91_RAMS_91,0,0,0,1
242,RAMS_93_RAMS_93,RAMS_93_RAMS_93,0,0,0,2
243,RAMS_95_RAMS_95,RAMS_95_RAMS_95,0,0,0,1


In [17]:
RAMS_pheno = openfile("/exports/reum/knevel_lab/RAMS/clinical/pheno.csv")
RAMS_pheno

,fid_anon,iid_anon,fupno,start_dose_mtx,steroids,vaspat,tjc28_value,sjc28_value,DAS28_CRP,sex,age_baseline,disease_duration_at_baseline_mths,logCRP,smoke_baseline
0,RAMS_1,RAMS_1,6 Mo,NaN,No,78.0,27.0,16.0,7.087547,NaN,NaN,NaN,2.793616,NaN
1,RAMS_1,RAMS_1,Baseline,15.0,No,77.0,24.0,11.0,6.549305,F,56.92845,17.311480,2.331172,current
2,RAMS_3,RAMS_3,Baseline,15.0,No,50.0,5.0,7.0,4.952939,M,43.01267,14.032790,3.610918,current
3,RAMS_3,RAMS_3,3 Mo,NaN,NaN,30.0,3.0,2.0,3.695989,NaN,NaN,NaN,2.639057,NaN
4,RAMS_3,RAMS_3,6 Mo,NaN,No,90.0,0.0,0.0,3.378795,NaN,NaN,NaN,3.218876,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,RAMS_526,RAMS_526,Baseline,7.5,Yes,26.0,2.0,0.0,3.047825,F,48.36700,4.524590,2.588516,never
913,RAMS_527,RAMS_527,3 Mo,NaN,NaN,24.0,5.0,0.0,3.193232,NaN,NaN,NaN,1.791759,NaN
914,RAMS_527,RAMS_527,Baseline,10.0,No,50.0,16.0,10.0,5.430471,M,77.39542,7.606557,1.791759,never
915,RAMS_527,RAMS_527,12 Mo,NaN,No,28.0,0.0,0.0,1.997033,NaN,NaN,NaN,1.791759,NaN


In [18]:
RAMS_pheno['CRP']=RAMS_pheno['logCRP'].map(np.exp)

In [20]:
RAMS_pheno['2COMPDAS']=TwocompDAS(RAMS_pheno['sjc28_value'],RAMS_pheno['CRP'])

In [23]:
RAMS_pheno['id'] = RAMS_pheno['fid_anon']+"_"+RAMS_pheno['iid_anon']

In [24]:
RAMS_pheno

,fid_anon,iid_anon,fupno,start_dose_mtx,steroids,vaspat,tjc28_value,sjc28_value,DAS28_CRP,sex,age_baseline,disease_duration_at_baseline_mths,logCRP,smoke_baseline,CRP,2COMPDAS,id
0,RAMS_1,RAMS_1,6 Mo,NaN,No,78.0,27.0,16.0,7.087547,NaN,NaN,NaN,2.793616,NaN,16.339999,5.711810,RAMS_1_RAMS_1
1,RAMS_1,RAMS_1,Baseline,15.0,No,77.0,24.0,11.0,6.549305,F,56.92845,17.311480,2.331172,current,10.289994,4.770975,RAMS_1_RAMS_1
2,RAMS_3,RAMS_3,Baseline,15.0,No,50.0,5.0,7.0,4.952939,M,43.01267,14.032790,3.610918,current,37.000003,4.828303,RAMS_3_RAMS_3
3,RAMS_3,RAMS_3,3 Mo,NaN,NaN,30.0,3.0,2.0,3.695989,NaN,NaN,NaN,2.639057,NaN,13.999995,3.039043,RAMS_3_RAMS_3
4,RAMS_3,RAMS_3,6 Mo,NaN,No,90.0,0.0,0.0,3.378795,NaN,NaN,NaN,3.218876,NaN,25.000004,1.954858,RAMS_3_RAMS_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,RAMS_526,RAMS_526,Baseline,7.5,Yes,26.0,2.0,0.0,3.047825,F,48.36700,4.524590,2.588516,never,13.310005,1.596575,RAMS_526_RAMS_526
913,RAMS_527,RAMS_527,3 Mo,NaN,NaN,24.0,5.0,0.0,3.193232,NaN,NaN,NaN,1.791759,NaN,5.999997,1.167546,RAMS_527_RAMS_527
914,RAMS_527,RAMS_527,Baseline,10.0,No,50.0,16.0,10.0,5.430471,M,77.39542,7.606557,1.791759,never,5.999997,4.329824,RAMS_527_RAMS_527
915,RAMS_527,RAMS_527,12 Mo,NaN,No,28.0,0.0,0.0,1.997033,NaN,NaN,NaN,1.791759,NaN,5.999997,1.167546,RAMS_527_RAMS_527


In [27]:
merge_RAMS = pd.merge(RAMS_fam,RAMS_pheno[RAMS_pheno['fupno']=='6 Mo'],how='inner',left_on=0,right_on='id')
merge_RAMS

,0,1,2,3,4,5,fid_anon,iid_anon,fupno,start_dose_mtx,steroids,vaspat,tjc28_value,sjc28_value,DAS28_CRP,sex,age_baseline,disease_duration_at_baseline_mths,logCRP,smoke_baseline,CRP,2COMPDAS,id
0,RAMS_1_RAMS_1,RAMS_1_RAMS_1,0,0,0,1,RAMS_1,RAMS_1,6 Mo,NaN,No,78.0,27.0,16.0,7.087547,NaN,NaN,NaN,2.793616,NaN,16.339999,5.711810,RAMS_1_RAMS_1
1,RAMS_10_RAMS_10,RAMS_10_RAMS_10,0,0,0,1,RAMS_10,RAMS_10,6 Mo,NaN,No,100.0,21.0,13.0,6.731171,NaN,NaN,NaN,2.209373,NaN,9.110003,4.993666,RAMS_10_RAMS_10
2,RAMS_100_RAMS_100,RAMS_100_RAMS_100,0,0,0,1,RAMS_100,RAMS_100,6 Mo,NaN,NaN,60.0,4.0,4.0,4.164221,NaN,NaN,NaN,1.900614,NaN,6.690001,3.223953,RAMS_100_RAMS_100
3,RAMS_102_RAMS_102,RAMS_102_RAMS_102,0,0,0,1,RAMS_102,RAMS_102,6 Mo,NaN,No,50.0,4.0,2.0,3.571480,NaN,NaN,NaN,1.098612,NaN,2.999999,2.245990,RAMS_102_RAMS_102
4,RAMS_103_RAMS_103,RAMS_103_RAMS_103,0,0,0,1,RAMS_103,RAMS_103,6 Mo,NaN,No,75.0,0.0,1.0,2.639480,NaN,NaN,NaN,0.970779,NaN,2.640000,1.775190,RAMS_103_RAMS_103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,RAMS_89_RAMS_89,RAMS_89_RAMS_89,0,0,0,1,RAMS_89,RAMS_89,6 Mo,NaN,Yes,37.0,2.0,0.0,3.135811,NaN,NaN,NaN,2.405142,NaN,11.080004,1.494931,RAMS_89_RAMS_89
241,RAMS_91_RAMS_91,RAMS_91_RAMS_91,0,0,0,1,RAMS_91,RAMS_91,6 Mo,NaN,Yes,71.0,26.0,2.0,6.567739,NaN,NaN,NaN,3.784190,NaN,44.000016,3.698211,RAMS_91_RAMS_91
242,RAMS_93_RAMS_93,RAMS_93_RAMS_93,0,0,0,2,RAMS_93,RAMS_93,6 Mo,NaN,No,4.0,0.0,0.0,1.284808,NaN,NaN,NaN,0.746688,NaN,2.110000,0.680774,RAMS_93_RAMS_93
243,RAMS_95_RAMS_95,RAMS_95_RAMS_95,0,0,0,1,RAMS_95,RAMS_95,6 Mo,NaN,No,87.0,4.0,1.0,4.278013,NaN,NaN,NaN,1.944481,NaN,6.990003,2.246915,RAMS_95_RAMS_95


In [28]:
# 245 samples
merge_RAMS.loc[:,[0,1,2,3,4,'2COMPDAS']].to_csv("/exports/reum/CKe/Pipelinerun_withPC/TwoCompDAS/RAMS_merged_2cDAS.fam",sep='\t',index=False,header=False)

### Vienna

In [26]:
vienna_fam = pd.read_csv('./Pipelinerun_withPC/Vienna/mergedChr/vienna_merged.fam',sep='\s',header=None)
vienna_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,10_1130820744,10_1130820744,0,0,0,2
1,1012_1041715812,1012_1041715812,0,0,0,2
2,1014_1131619222,1014_1131619222,0,0,0,2
3,1015_1150204122,1015_1150204122,0,0,0,1
4,1017_1041664226,1017_1041664226,0,0,0,1
...,...,...,...,...,...,...
401,967_1153937793,967_1153937793,0,0,0,-9
402,988_1153942083,988_1153942083,0,0,0,1
403,989_1085427379,989_1085427379,0,0,0,2
404,990_1150146097,990_1150146097,0,0,0,2


In [27]:
Vienna = openfile("/exports/reum/knevel_lab/Vienna/clinical/Rachel_data_ready_to deliver.csv",sep=';')
Vienna

,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge
0,clin,2,4/11/2007,1,4/11/2007,F,"49,5",4/1/2007,0,4/11/2007,0,0,nein,False,1,ja,False,50,True,",3",16,36,34,41,",25",4,7,7,4,12,"1,1","4,48643283976402",MDA,"4,00468670419933","18,3",MDA,18,MDA,0,0,8/2/2007,20,,,Mtx,1,8,4/11/2007,12/6/2007,1,MTX,0,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,1,BOTH,1085421526,3
1,both,2,4/26/2007,2,4/11/2007,F,"49,6",4/1/2007,",1",4/11/2007,0,0,nein,False,1,ja,False,5,True,",32",17,32,31,46,",625",4,9,10,4,12,"1,4","4,67124934083935",MDA,"4,16463042910416","19,62",MDA,"19,3",MDA,0,0,8/2/2007,20,,,Mtx,2,8,4/11/2007,12/6/2007,1,MTX,0,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3
2,clin,2,8/2/2007,3,4/11/2007,F,"49,8",4/1/2007,",3",4/11/2007,",3",0,nein,False,1,ja,False,5,True,",34",16,24,6,31,",5",0,3,3,0,,"1,7","3,24676055780642",MDA,"2,79932608697129","6,34",LDA,6,LDA,0,0,8/2/2007,15,,,Mtx,3,8,4/11/2007,12/6/2007,1,MTX,3,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3
3,both,2,12/6/2007,4,4/11/2007,F,"50,2",4/1/2007,",7",4/11/2007,",7",0,nein,False,1,ja,False,5,True,",42",22,31,3,33,",625",0,4,4,0,12,1,"3,71772971735082",MDA,"3,10751710521146","7,82",LDA,"7,4",LDA,0,0,8/2/2007,,,,,1,9,12/6/2007,9/1/2008,2,,7,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3
4,both,4,3/14/2007,1,3/14/2007,W,"51,3",,,,,2,,False,1,ja,False,"7,5",False,",68",54,74,8,72,",625",0,6,6,0,12,"1,3","5,20000308855357",HDA,"4,10719880008898","14,88",MDA,"14,2",MDA,0,0,3/14/2007,10,,,Mtx,1,"35,1",3/14/2007,1/29/2010,1,MTX,0,1,W,GG,GG,AA,AG,GG,AT,CT,TT,DRB1*07:01:01:01,DRB1*12:01:01Â,0,3,1,BOTH,1031467068,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,clin,4239,5/21/2014,2,2/14/2014,W,"45,4",2/1/2012,"2,3",2/14/2014,",3",1,ja,False,1,ja,False,"6,25",False,"1,09",28,56,9,24,"1,375",1,1,2,1,77,,"3,95654315712264",MDA,"3,47555382404229","9,59",LDA,"8,5",LDA,1,1,2/14/2014,25,,,Mtx,2,"15,1",2/14/2014,5/13/2015,1,MTX,3,,,,,,,,,,,,,,1,0,BOTH,1095887411,3
2209,clin,4239,6/25/2014,3,2/14/2014,W,"45,5",2/1/2012,"2,4",2/14/2014,",4",1,ja,False,0,nein,False,,False,",53",19,0,0,0,0,0,0,0,0,38,,"2,06110728541651",REM,"1,6225978680231",",53",REM,0,REM,1,1,2/14/2014,25,,,Mtx,3,"15,1",2/14/2014,5/13/2015,1,MTX,4,,,,,,,,,,,,,,1,0,BOTH,1095887411,3
2210,clin,4239,9/17/2014,4,2/14/2014,W,"45,7",2/1/2012,"2,6",2/14/2014,",6",1,ja,False,0,nein,False,,False,",67",21,34,0,36,",625",0,2,2,0,88,,"3,39912530133533",MDA,"2,9627989133184","6,07",LDA,"5,4",LDA,1,1,2/14/2014,25,,,Mtx,4,"15,1",2/14/2014,5/13/2015,1,MTX,7,,,,,,,,,,,,,,1,0,BOTH,1095887411,3
2211,clin,4239,1/14/2015,5,2/14/2014,W,"46,1",2/1/2012,3,2/14/2014,",9",1,ja,False,0,nein,False,,False,",25",21,52,0,1,",125",0,0,0,0,,,"2,8591657064064",LDA,"2,13899466865833","5,45",LDA,"5,2",LDA,1,1,2/14/2014,25,,,Mtx,5,"15,1",2/14/2014,5/13/2015,1,MTX,11,,,,,,,,,,,,,,1,0,BOTH,1095887411,3


In [11]:
search(Vienna.columns)

100%|████████████████████████████████████████| 70/70 [00:00<00:00, 44552.55it/s]

Found pattern crp in column num 19 CRP
Found pattern sjc in column num 25 SJC28
Found pattern sjc in column num 28 SJC32
Found pattern crp in column num 33 DAS_crp
Found pattern sdai in column num 34 SDAI
Found pattern sdai in column num 35 SDAI_S
Found pattern cdai in column num 36 CDAI
Found pattern cdai in column num 37 CDAI_S
Found pattern mtx in column num 41 MTX_DOSIS


['CRP',
 'SJC28',
 'SJC32',
 'DAS_crp',
 'SDAI',
 'SDAI_S',
 'CDAI',
 'CDAI_S',
 'MTX_DOSIS']

In [28]:
Vienna['CRP'] = Vienna['CRP'].str.replace(',','.').replace(' ',0).astype(np.float)
Vienna['followup'] = Vienna['followup'].map(todatetime)
Vienna['Eintrt_datum'] = Vienna['Eintrt_datum'].map(todatetime)

/tmp/ipykernel_1807674/2762481165.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Vienna['CRP'] = Vienna['CRP'].str.replace(',','.').replace(' ',0).astype(np.float)
/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [29]:
Vienna['SJC28'] = Vienna['SJC28'].str.replace(',','.').replace(' ',0).astype(np.float)

/tmp/ipykernel_1807674/616683817.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Vienna['SJC28'] = Vienna['SJC28'].str.replace(',','.').replace(' ',0).astype(np.float)


In [30]:
Vienna

,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge
0,clin,2,2007-04-11,1,2007-04-11,F,"49,5",4/1/2007,0,4/11/2007,0,0,nein,False,1,ja,False,50,True,0.30,16,36,34,41,",25",4.0,7,7,4,12,"1,1","4,48643283976402",MDA,"4,00468670419933","18,3",MDA,18,MDA,0,0,8/2/2007,20,,,Mtx,1,8,4/11/2007,12/6/2007,1,MTX,0,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,1,BOTH,1085421526,3
1,both,2,2007-04-26,2,2007-04-11,F,"49,6",4/1/2007,",1",4/11/2007,0,0,nein,False,1,ja,False,5,True,0.32,17,32,31,46,",625",4.0,9,10,4,12,"1,4","4,67124934083935",MDA,"4,16463042910416","19,62",MDA,"19,3",MDA,0,0,8/2/2007,20,,,Mtx,2,8,4/11/2007,12/6/2007,1,MTX,0,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3
2,clin,2,2007-08-02,3,2007-04-11,F,"49,8",4/1/2007,",3",4/11/2007,",3",0,nein,False,1,ja,False,5,True,0.34,16,24,6,31,",5",0.0,3,3,0,,"1,7","3,24676055780642",MDA,"2,79932608697129","6,34",LDA,6,LDA,0,0,8/2/2007,15,,,Mtx,3,8,4/11/2007,12/6/2007,1,MTX,3,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3
3,both,2,2007-12-06,4,2007-04-11,F,"50,2",4/1/2007,",7",4/11/2007,",7",0,nein,False,1,ja,False,5,True,0.42,22,31,3,33,",625",0.0,4,4,0,12,1,"3,71772971735082",MDA,"3,10751710521146","7,82",LDA,"7,4",LDA,0,0,8/2/2007,,,,,1,9,12/6/2007,9/1/2008,2,,7,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3
4,both,4,2007-03-14,1,2007-03-14,W,"51,3",,,,,2,,False,1,ja,False,"7,5",False,0.68,54,74,8,72,",625",0.0,6,6,0,12,"1,3","5,20000308855357",HDA,"4,10719880008898","14,88",MDA,"14,2",MDA,0,0,3/14/2007,10,,,Mtx,1,"35,1",3/14/2007,1/29/2010,1,MTX,0,1,W,GG,GG,AA,AG,GG,AT,CT,TT,DRB1*07:01:01:01,DRB1*12:01:01Â,0,3,1,BOTH,1031467068,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,clin,4239,2014-05-21,2,2014-02-14,W,"45,4",2/1/2012,"2,3",2/14/2014,",3",1,ja,False,1,ja,False,"6,25",False,1.09,28,56,9,24,"1,375",1.0,1,2,1,77,,"3,95654315712264",MDA,"3,47555382404229","9,59",LDA,"8,5",LDA,1,1,2/14/2014,25,,,Mtx,2,"15,1",2/14/2014,5/13/2015,1,MTX,3,,,,,,,,,,,,,,1,0,BOTH,1095887411,3
2209,clin,4239,2014-06-25,3,2014-02-14,W,"45,5",2/1/2012,"2,4",2/14/2014,",4",1,ja,False,0,nein,False,,False,0.53,19,0,0,0,0,0.0,0,0,0,38,,"2,06110728541651",REM,"1,6225978680231",",53",REM,0,REM,1,1,2/14/2014,25,,,Mtx,3,"15,1",2/14/2014,5/13/2015,1,MTX,4,,,,,,,,,,,,,,1,0,BOTH,1095887411,3
2210,clin,4239,2014-09-17,4,2014-02-14,W,"45,7",2/1/2012,"2,6",2/14/2014,",6",1,ja,False,0,nein,False,,False,0.67,21,34,0,36,",625",0.0,2,2,0,88,,"3,39912530133533",MDA,"2,9627989133184","6,07",LDA,"5,4",LDA,1,1,2/14/2014,25,,,Mtx,4,"15,1",2/14/2014,5/13/2015,1,MTX,7,,,,,,,,,,,,,,1,0,BOTH,1095887411,3
2211,clin,4239,2015-01-14,5,2014-02-14,W,"46,1",2/1/2012,3,2/14/2014,",9",1,ja,False,0,nein,False,,False,0.25,21,52,0,1,",125",0.0,0,0,0,,,"2,8591657064064",LDA,"2,13899466865833","5,45",LDA,"5,2",LDA,1,1,2/14/2014,25,,,Mtx,5,"15,1",2/14/2014,5/13/2015,1,MTX,11,,,,,,,,,,,,,,1,0,BOTH,1095887411,3


In [31]:
Vienna['2COMPDAS']=TwocompDAS(Vienna['SJC28'],Vienna['CRP'])

In [32]:
Vienna_select = Vienna[(Vienna['followup']-Vienna['Eintrt_datum']<=timedelta(180))]
Vienna_select

,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge,2COMPDAS
0,clin,2,2007-04-11,1,2007-04-11,F,"49,5",4/1/2007,0,4/11/2007,0,0,nein,False,1,ja,False,50,True,0.30,16,36,34,41,",25",4.0,7,7,4,12,"1,1","4,48643283976402",MDA,"4,00468670419933","18,3",MDA,18,MDA,0,0,8/2/2007,20,,,Mtx,1,8,4/11/2007,12/6/2007,1,MTX,0,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,1,BOTH,1085421526,3,2.157419
1,both,2,2007-04-26,2,2007-04-11,F,"49,6",4/1/2007,",1",4/11/2007,0,0,nein,False,1,ja,False,5,True,0.32,17,32,31,46,",625",4.0,9,10,4,12,"1,4","4,67124934083935",MDA,"4,16463042910416","19,62",MDA,"19,3",MDA,0,0,8/2/2007,20,,,Mtx,2,8,4/11/2007,12/6/2007,1,MTX,0,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3,2.166579
2,clin,2,2007-08-02,3,2007-04-11,F,"49,8",4/1/2007,",3",4/11/2007,",3",0,nein,False,1,ja,False,5,True,0.34,16,24,6,31,",5",0.0,3,3,0,,"1,7","3,24676055780642",MDA,"2,79932608697129","6,34",LDA,6,LDA,0,0,8/2/2007,15,,,Mtx,3,8,4/11/2007,12/6/2007,1,MTX,3,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3,0.175602
4,both,4,2007-03-14,1,2007-03-14,W,"51,3",,,,,2,,False,1,ja,False,"7,5",False,0.68,54,74,8,72,",625",0.0,6,6,0,12,"1,3","5,20000308855357",HDA,"4,10719880008898","14,88",MDA,"14,2",MDA,0,0,3/14/2007,10,,,Mtx,1,"35,1",3/14/2007,1/29/2010,1,MTX,0,1,W,GG,GG,AA,AG,GG,AT,CT,TT,DRB1*07:01:01:01,DRB1*12:01:01Â,0,3,1,BOTH,1031467068,3,0.311276
5,both,4,2007-07-18,2,2007-03-14,W,"51,6",,,,,2,,False,0,nein,False,,False,0.28,27,87,1,55,"1,25",0.0,0,0,0,12,1,"3,52508580620303",MDA,"2,65860038402364","9,08",LDA,"8,8",LDA,0,0,3/14/2007,20,,,Mtx,2,"35,1",3/14/2007,1/29/2010,1,MTX,4,1,W,GG,GG,AA,AG,GG,AT,CT,TT,DRB1*07:01:01:01,DRB1*12:01:01Â,0,3,0,BOTH,1031467068,3,0.148116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,clin,4236,2005-05-19,2,2005-03-17,F,"67,8",3/1/2003,"2,2",3/17/2005,",2",0,nein,False,1,ja,False,5,True,1.21,49,0,0,0,0,0.0,0,0,0,,,"2,72427420867744",LDA,"1,88614040287456","1,21",REM,0,REM,0,0,3/17/2005,,,,Resochin,2,"4,6",3/17/2005,8/2/2005,1,csDMARD,2,,W,GG,GG,,AA,GG,TT,TT,CT,DRB1*04:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1031466324,3,0.475796
2204,clin,4236,2005-08-02,3,2005-03-17,F,68,3/1/2003,"2,4",3/17/2005,",4",0,nein,True,1,ja,False,5,True,1.38,18,0,0,28,0,0.0,0,0,0,12,,"2,02326023052732",REM,"1,93006578507722","1,38",REM,0,REM,0,0,3/17/2005,10,,,Mtx,1,"69,8",8/2/2005,4/26/2011,2,MTX,4,,W,GG,GG,,AA,GG,TT,TT,CT,DRB1*04:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1031466324,3,0.520260
2207,both,4239,2014-02-14,1,2014-02-14,W,"45,2",2/1/2012,2,2/14/2014,0,1,ja,False,2,,False,,False,0.33,24,17,9,5,",5",2.0,1,1,2,123,,"3,41861747870803",MDA,"2,67908120563629","5,93",LDA,"5,6",LDA,1,1,2/14/2014,20,,,Mtx,1,"15,1",2/14/2014,5/13/2015,1,MTX,0,,,,,,,,,,,,,,1,1,BOTH,1095887411,3,1.585321
2208,clin,4239,2014-05-21,2,2014-02-14,W,"45,4",2/1/2012,"2,3",2/14/2014,",3",1,ja,False,1,ja,False,"6,25",False,1.09,28,56,9,24,"1,375",1.0,1,2,1,77,,"3,95654315712264",MDA,"3,47555382404229","9,59",LDA,"8,5",LDA,1,1,2/14/2014,25,,,Mtx,2,"15,1",2/14/2014,5/13/2015,1,MTX,3,,,,,,,,,,,,,,1,0,BOTH,1095887411,3,1.442298


In [33]:
Vienna_select['CDAI_S'] = Vienna_select['CDAI_S'].map({"MDA":1,"LDA":1,"HDA":1,"REM":2," ":0})
Vienna_select['SDAI_S'] = Vienna_select['SDAI_S'].map({"MDA":1,"LDA":1,"HDA":1,"REM":2," ":0})

/tmp/ipykernel_1807674/3569286012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vienna_select['CDAI_S'] = Vienna_select['CDAI_S'].map({"MDA":1,"LDA":1,"HDA":1,"REM":2," ":0})
/tmp/ipykernel_1807674/3569286012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Vienna_select['SDAI_S'] = Vienna_select['SDAI_S'].map({"MDA":1,"LDA":1,"HDA":1,"REM":2," ":0})


In [34]:
Vienna_select

,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge,2COMPDAS
0,clin,2,2007-04-11,1,2007-04-11,F,"49,5",4/1/2007,0,4/11/2007,0,0,nein,False,1,ja,False,50,True,0.30,16,36,34,41,",25",4.0,7,7,4,12,"1,1","4,48643283976402",MDA,"4,00468670419933","18,3",1,18,1,0,0,8/2/2007,20,,,Mtx,1,8,4/11/2007,12/6/2007,1,MTX,0,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,1,BOTH,1085421526,3,2.157419
1,both,2,2007-04-26,2,2007-04-11,F,"49,6",4/1/2007,",1",4/11/2007,0,0,nein,False,1,ja,False,5,True,0.32,17,32,31,46,",625",4.0,9,10,4,12,"1,4","4,67124934083935",MDA,"4,16463042910416","19,62",1,"19,3",1,0,0,8/2/2007,20,,,Mtx,2,8,4/11/2007,12/6/2007,1,MTX,0,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3,2.166579
2,clin,2,2007-08-02,3,2007-04-11,F,"49,8",4/1/2007,",3",4/11/2007,",3",0,nein,False,1,ja,False,5,True,0.34,16,24,6,31,",5",0.0,3,3,0,,"1,7","3,24676055780642",MDA,"2,79932608697129","6,34",1,6,1,0,0,8/2/2007,15,,,Mtx,3,8,4/11/2007,12/6/2007,1,MTX,3,,W,GG,GG,AG,AA,GG,TT,TT,CT,DRB1*01:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1085421526,3,0.175602
4,both,4,2007-03-14,1,2007-03-14,W,"51,3",,,,,2,,False,1,ja,False,"7,5",False,0.68,54,74,8,72,",625",0.0,6,6,0,12,"1,3","5,20000308855357",HDA,"4,10719880008898","14,88",1,"14,2",1,0,0,3/14/2007,10,,,Mtx,1,"35,1",3/14/2007,1/29/2010,1,MTX,0,1,W,GG,GG,AA,AG,GG,AT,CT,TT,DRB1*07:01:01:01,DRB1*12:01:01Â,0,3,1,BOTH,1031467068,3,0.311276
5,both,4,2007-07-18,2,2007-03-14,W,"51,6",,,,,2,,False,0,nein,False,,False,0.28,27,87,1,55,"1,25",0.0,0,0,0,12,1,"3,52508580620303",MDA,"2,65860038402364","9,08",1,"8,8",1,0,0,3/14/2007,20,,,Mtx,2,"35,1",3/14/2007,1/29/2010,1,MTX,4,1,W,GG,GG,AA,AG,GG,AT,CT,TT,DRB1*07:01:01:01,DRB1*12:01:01Â,0,3,0,BOTH,1031467068,3,0.148116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,clin,4236,2005-05-19,2,2005-03-17,F,"67,8",3/1/2003,"2,2",3/17/2005,",2",0,nein,False,1,ja,False,5,True,1.21,49,0,0,0,0,0.0,0,0,0,,,"2,72427420867744",LDA,"1,88614040287456","1,21",2,0,2,0,0,3/17/2005,,,,Resochin,2,"4,6",3/17/2005,8/2/2005,1,csDMARD,2,,W,GG,GG,,AA,GG,TT,TT,CT,DRB1*04:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1031466324,3,0.475796
2204,clin,4236,2005-08-02,3,2005-03-17,F,68,3/1/2003,"2,4",3/17/2005,",4",0,nein,True,1,ja,False,5,True,1.38,18,0,0,28,0,0.0,0,0,0,12,,"2,02326023052732",REM,"1,93006578507722","1,38",2,0,2,0,0,3/17/2005,10,,,Mtx,1,"69,8",8/2/2005,4/26/2011,2,MTX,4,,W,GG,GG,,AA,GG,TT,TT,CT,DRB1*04:01:01,DRB1*15:01:01:01Â,1,3,0,BOTH,1031466324,3,0.520260
2207,both,4239,2014-02-14,1,2014-02-14,W,"45,2",2/1/2012,2,2/14/2014,0,1,ja,False,2,,False,,False,0.33,24,17,9,5,",5",2.0,1,1,2,123,,"3,41861747870803",MDA,"2,67908120563629","5,93",1,"5,6",1,1,1,2/14/2014,20,,,Mtx,1,"15,1",2/14/2014,5/13/2015,1,MTX,0,,,,,,,,,,,,,,1,1,BOTH,1095887411,3,1.585321
2208,clin,4239,2014-05-21,2,2014-02-14,W,"45,4",2/1/2012,"2,3",2/14/2014,",3",1,ja,False,1,ja,False,"6,25",False,1.09,28,56,9,24,"1,375",1.0,1,2,1,77,,"3,95654315712264",MDA,"3,47555382404229","9,59",1,"8,5",1,1,1,2/14/2014,25,,,Mtx,2,"15,1",2/14/2014,5/13/2015,1,MTX,3,,,,,,,,,,,,,,1,0,BOTH,1095887411,3,1.442298


In [35]:
vienna_fam['id'] = vienna_fam[0].str.split("_",expand=True)[1]
vienna_fam

,0,1,2,3,4,5,id
0,10_1130820744,10_1130820744,0,0,0,2,1130820744
1,1012_1041715812,1012_1041715812,0,0,0,2,1041715812
2,1014_1131619222,1014_1131619222,0,0,0,2,1131619222
3,1015_1150204122,1015_1150204122,0,0,0,1,1150204122
4,1017_1041664226,1017_1041664226,0,0,0,1,1041664226
...,...,...,...,...,...,...,...
401,967_1153937793,967_1153937793,0,0,0,-9,1153937793
402,988_1153942083,988_1153942083,0,0,0,1,1153942083
403,989_1085427379,989_1085427379,0,0,0,2,1085427379
404,990_1150146097,990_1150146097,0,0,0,2,1150146097


In [36]:
vienna_merge = pd.merge(vienna_fam, Vienna_select, left_on="id",right_on="SampleID_last",how='left')
vienna_merge

,0,1,2,3,4,5,id,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge,2COMPDAS
0,10_1130820744,10_1130820744,0,0,0,2,1130820744,both,3147.0,2011-09-27,1.0,2011-09-27,W,"41,6",5/1/2011,",4",12/15/2011,"-,2",0.0,nein,False,2,,False,,True,1.60,24,17,14,6,,3.0,3,3,4,59,157,"3,91756035960142",MDA,"3,67287948221809","10,7",1.0,"9,1",1.0,1.0,1,12/15/2011,,,,,,"2,6",9/27/2011,12/15/2011,,,0.0,1,W,GG,GG,GG,AA,GG,AA,TT,CT,DRB1*01:01:01,DRB1*13:02:01Â,1,3.0,1.0,BOTH,1130820744,3.0,2.305358
1,10_1130820744,10_1130820744,0,0,0,2,1130820744,clin,3147.0,2011-12-15,2.0,2011-09-27,W,"41,9",5/1/2011,",6",12/15/2011,0,0.0,nein,False,1,ja,True,"6,25",True,0.63,20,37,8,29,,2.0,5,6,4,128,163,"4,26319045635214",MDA,"3,84181263019991","12,13",1.0,"11,5",1.0,1.0,1,12/15/2011,10,,,Mtx,1,"8,6",12/15/2011,8/28/2012,1,MTX,2.0,1,W,GG,GG,GG,AA,GG,AA,TT,CT,DRB1*01:01:01,DRB1*13:02:01Â,1,3.0,0.0,BOTH,1130820744,3.0,1.707362
2,10_1130820744,10_1130820744,0,0,0,2,1130820744,both,3147.0,2012-03-20,3.0,2011-09-27,W,"42,1",5/1/2011,",9",12/15/2011,",3",0.0,nein,False,0,nein,False,,True,0.48,21,0,0,0,0,0.0,0,0,0,38,78,"2,1311657064064",REM,"1,59282885031885",",48",2.0,0,2.0,1.0,1,12/15/2011,20,,,Mtx,2,"8,6",12/15/2011,8/28/2012,1,MTX,5.0,1,W,GG,GG,GG,AA,GG,AA,TT,CT,DRB1*01:01:01,DRB1*13:02:01Â,1,3.0,0.0,BOTH,1130820744,3.0,0.235225
3,1012_1041715812,1012_1041715812,0,0,0,2,1041715812,clin,1614.0,2002-08-22,1.0,2002-08-22,F,"56,3",10/1/2001,",9",6/1/2002,",2",1.0,ja,False,0,nein,False,,True,0.50,8,45,37,45,"1,875",5.0,5,5,5,883,,"3,96390618027571",MDA,"4,11333051002192","18,7",1.0,"18,2",1.0,1.0,1,8/22/2002,,,2000,Salazopyrin,1,17,8/22/2002,1/13/2004,1,csDMARD,0.0,,,,,,,,,,,,,,1.0,1.0,BOTH,1041715812,3.0,2.479347
4,1012_1041715812,1012_1041715812,0,0,0,2,1041715812,clin,1614.0,2003-02-07,2.0,2002-08-22,F,"56,8",10/1/2001,"1,4",6/1/2002,",7",1.0,ja,False,0,nein,False,,False,0.50,7,33,23,35,",25",2.0,3,3,2,291,,"3,19006535404176",LDA,"3,43296165862514","11,1",1.0,"10,6",1.0,1.0,1,8/22/2002,,,3000,Salazopyrin,2,17,8/22/2002,1/13/2004,1,csDMARD,5.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1041715812,3.0,1.657493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,990_1150146097,990_1150146097,0,0,0,2,1150146097,clin,3357.0,1998-01-21,1.0,1998-01-21,F,"25,2",4/30/1997,",7",9/1/1997,",4",2.0,,False,1,ja,False,5,True,0.50,13,12,9,3,0,0.0,1,1,0,12,,"2,52346455022308",REM,"2,3330334089221","3,6",1.0,"3,1",1.0,0.0,0,3/1/1997,,,3000,Salazopyrin,1,"28,5",1/21/1998,5/26/2000,1,csDMARD,0.0,,W,AG,GG,GG,GG,GG,AT,CT,CT,DRB1*04:04:01,DRB1*08:01:01Â,1,3.0,1.0,BOTH,1150146097,3.0,0.243279
1022,990_1150146097,990_1150146097,0,0,0,2,1150146097,clin,3357.0,1998-03-13,2.0,1998-01-21,F,"25,4",4/30/1997,",9",9/1/1997,",5",2.0,,False,1,ja,False,"2,5",False,0.50,6,10,4,0,",375",0.0,0,0,0,,,"1,39423162845964",REM,"1,7450334089221","1,9",2.0,"1,4",2.0,0.0,0,3/1/1997,,,2500,Salazopyrin,2,"28,5",1/21/1998,5/26/2000,1,csDMARD,1.0,,W,AG,GG,GG,GG,GG,AT,CT,CT,DRB1*04:04:01,DRB1*08:01:01Â,1,3.0,0.0,BOTH,1150146097,3.0,0.243279
1023,991_1131615031,991_1131615031,0,0,0,1,1131615031,both,2100.0,2012-07-24,1.0,2012-07-24,W,"42,2",5/24/2012,",2",8/20/2012,"-,1",1.0,ja,False,1,ja,True,"6,25",False,1.05,21,58,45,56,",25",4.0,5,5,

In [37]:
vienna_merge = vienna_merge.drop_duplicates('id',keep='last')
vienna_merge

,0,1,2,3,4,5,id,desc,patient,followup,visit_nr,Eintrt_datum,sex,age,Beschwerdebeginn,Beschwerdedauer,Erstdiagnose,Krankenheitsdauer,Nikotin,Nikotin_text,Analgetika,Kortison,Kortison_merkmal,KortisonStoss,KortisonDosis,NSAIDs,CRP,ESR,PGA,EGA,PAIN,HAQ,SJC28,TJC28,TJC32,SJC32,RF,CCP,DAS_esr,DAS_ESR_S,DAS_crp,SDAI,SDAI_S,CDAI,CDAI_S,rf_pos,ccp_pos,DMARD_Start,MTX_DOSIS,Arava_DOSIS,Salazopyrin_DOSIS,trtment,tr_visit,tr_dur,tr_b,tr_e,trt_course,DMARD__cat,care_dur,RRECEPT_B,locality,PTPN_22,STAT_4,CTLA_4,TRAF1_C5,TNFAIP3,AFF3,IRF5,PADI4,hla1,hla2,shared_epitope,@___merge,filter__,discrib,SampleID_last,@_merge,2COMPDAS
2,10_1130820744,10_1130820744,0,0,0,2,1130820744,both,3147.0,2012-03-20,3.0,2011-09-27,W,"42,1",5/1/2011,",9",12/15/2011,",3",0.0,nein,False,0,nein,False,,True,0.48,21,0,0,0,0,0.0,0,0,0,38,78,"2,1311657064064",REM,"1,59282885031885",",48",2.0,0,2.0,1.0,1,12/15/2011,20,,,Mtx,2,"8,6",12/15/2011,8/28/2012,1,MTX,5.0,1,W,GG,GG,GG,AA,GG,AA,TT,CT,DRB1*01:01:01,DRB1*13:02:01Â,1,3.0,0.0,BOTH,1130820744,3.0,0.235225
4,1012_1041715812,1012_1041715812,0,0,0,2,1041715812,clin,1614.0,2003-02-07,2.0,2002-08-22,F,"56,8",10/1/2001,"1,4",6/1/2002,",7",1.0,ja,False,0,nein,False,,False,0.50,7,33,23,35,",25",2.0,3,3,2,291,,"3,19006535404176",LDA,"3,43296165862514","11,1",1.0,"10,6",1.0,1.0,1,8/22/2002,,,3000,Salazopyrin,2,17,8/22/2002,1/13/2004,1,csDMARD,5.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1041715812,3.0,1.657493
8,1014_1131619222,1014_1131619222,0,0,0,2,1131619222,both,326.0,2006-12-27,4.0,2006-08-23,W,"61,4",7/15/2006,",5",7/1/2006,",5",0.0,nein,False,1,ja,False,"9,375",True,0.11,8,34,9,40,",125",0.0,1,1,0,12,"1,3","2,49160907917588",REM,"2,26309744410258","5,41",1.0,"5,3",1.0,0.0,0,8/23/2006,20,,,Mtx,3,34,10/19/2006,8/4/2009,1,MTX,4.0,1,W,GG,GG,AG,GG,GG,TT,CT,CT,DRB1*04:04:01,DRB1*13:01:01Â,1,3.0,0.0,BOTH,1131619222,3.0,0.062616
11,1015_1150204122,1015_1150204122,0,0,0,1,1150204122,clin,2018.0,2005-08-19,3.0,2005-04-08,M,"44,9",12/14/2004,",7",4/8/2005,",4",1.0,ja,False,1,ja,False,"12,5",False,4.00,69,68,24,39,0,0.0,4,4,0,208,,"5,03587455321808",MDA,"4,36888594401355","17,2",1.0,"13,2",1.0,1.0,0,4/8/2005,25,,,Mtx,3,10,4/8/2005,2/3/2006,1,MTX,4.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1150204122,3.0,0.965663
15,1017_1041664226,1017_1041664226,0,0,0,1,1041664226,clin,2829.0,2008-07-16,4.0,2008-04-10,W,"76,9",1/15/2007,"1,5",5/1/2007,"1,2",0.0,nein,False,1,ja,False,"6,25",False,1.53,22,37,16,29,0,8.0,0,0,8,12,340,"3,47368931227975",MDA,"3,27477903374151","14,83",1.0,"13,3",1.0,1.0,1,4/10/2008,,20,,Arava,1,"6,2",7/16/2008,1/19/2009,2,csDMARD,3.0,5,W,GG,GG,AG,GG,GG,AT,CT,TT,DRB1*01:01:01,DRB1*04:01:01Â,2,3.0,0.0,BOTH,1041664226,3.0,3.385359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,967_1153937793,967_1153937793,0,0,0,-9,1153937793,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1017,988_1153942083,988_1153942083,0,0,0,1,1153942083,clin,2763.0,2014-06-06,2.0,2014-01-08,F,"50,5",6/1/2013,1,1/8/2014,",4",0.0,nein,False,0,nein,True,,False,0.94,,22,10,31,",625",2.0,4,4,2,12,",5",,,"3,6270298876775","10,14",1.0,"9,2",1.0,0.0,0,6/6/2014,10,,,Mtx,1,"13,6",6/6/2014,7/20/2015,1,MTX,4.0,,,,,,,,,,,,,,1.0,0.0,BOTH,1153942083,3.0,1.811826
1020,989_1085427379,989_1085427379,0,0,0,2,1085427379,clin,2894.0,2009-01-29,3.0,2008-09-25,W,42,12/1/2007,"1,2",9/25/2008,",3",0.0,nein,False,1,ja,False,5,False,0.00,,7,14,9,",125",2.0,3,3,2,,,,,,,0.0,"7,1",1.0,1.0,1,9/5/2008,25,,,Mtx,3,"125,7",9/25/2008,1/23/2019,1,MTX,4.0,2,W,GG,GT,AG,AA,GG,AA,CT,CC,DRB1*01:01:01,DRB1*16:01:01Â,1,3

In [38]:
vienna_merge['2COMPDAS'] = vienna_merge['2COMPDAS'].fillna(-9)
vienna_merge['SDAI_S'] = vienna_merge['SDAI_S'].fillna(-9)
vienna_merge['CDAI_S'] = vienna_merge['CDAI_S'].fillna(-9)

/tmp/ipykernel_1807674/4157333571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vienna_merge['2COMPDAS'] = vienna_merge['2COMPDAS'].fillna(-9)
/tmp/ipykernel_1807674/4157333571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vienna_merge['SDAI_S'] = vienna_merge['SDAI_S'].fillna(-9)
/tmp/ipykernel_1807674/4157333571.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [39]:
vienna_fam_2cDAS = vienna_merge.loc[:,[0,1,2,3,4,'2COMPDAS']]
vienna_fam_sDAI = vienna_merge.loc[:,[0,1,2,3,4,'SDAI_S']]
vienna_fam_cDAI = vienna_merge.loc[:,[0,1,2,3,4,'CDAI_S']]

In [47]:
vienna_fam_sDAI['SDAI_S'] = vienna_fam_sDAI['SDAI_S'].astype(int)
vienna_fam_cDAI['CDAI_S'] = vienna_fam_cDAI['CDAI_S'].astype(int)

In [48]:
#398 samples 
vienna_fam_2cDAS.to_csv("/exports/reum/CKe/Pipelinerun_withPC/TwoCompDAS/Vienna_merged_2cDAS.fam",sep='\t',index=False,header=False)
vienna_fam_sDAI.to_csv("/exports/reum/CKe/Pipelinerun_withPC/TwoCompDAS/Vienna_merged_sDAI.fam",sep='\t',index=False,header=False)
vienna_fam_cDAI.to_csv("/exports/reum/CKe/Pipelinerun_withPC/TwoCompDAS/Vienna_merged_cDAI.fam",sep='\t',index=False,header=False)

### Madrid

In [17]:
Madrid = openfile("/exports/reum/knevel_lab/Madrid/clinical/Clinical_data_GWAS_patients_HUP.dta")
Madrid_additional = openfile("/exports/reum/knevel_lab/Scripts_Samantha/GWAS_runscripts/GWAS_scripts/input_files_used/madrid/Additional clincal data pacients GWAS HUP.xls")

In [66]:
Madrid

,codigo2,tabaco,fr,fechaini,sexo,cort_v1,dosis_mtx_v1,dosis_leflu_v1,dosis_antipal_v1,dosis_szp_v1,dosis_oro_v1,dosis_csa_v1,a_tnf_v1,aba_v1,toci_v1,ritu_v1,cort_v2,dosis_mtx_v2,dosis_leflu_v2,dosis_antipal_v2,dosis_szp_v2,dosis_oro_v2,dosis_csa_v2,a_tnf_v2,aba_v2,toci_v2,ritu_v2,vgepac_v3,vgemed_v3,nad_v3,nat_v3,vsg_v3,pcr_v3,cort_v3,dosis_mtx_v3,dosis_leflu_v3,dosis_antipal_v3,dosis_szp_v3,dosis_oro_v3,dosis_csa_v3,a_tnf_v3,aba_v3,toci_v3,ritu_v3,das28_v3,das28pcr_v3,das28cat_v3,das28pcrcat_v3,sdai_v3,cdai_v3,cort_v4,edadinienf,tevol_v1,tevol_v3,ccpsn,hupi_v3,hupicat_v3,cortic_1yr
0,536,1.0,1,2012-08-15,Mujer,0.0,0.0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,14.0,0.0,0.0,29.0,0.30,0.0,17.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.525107,1.627066,Remision,Remision,2.900000,2.600000,0.0,65.180824,2.266667,14.400000,1.0,1.0,Remision,0.000
1,539,0.0,0,2012-08-20,Mujer,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,17.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,45.0,6.0,1.0,21.0,0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.510880,3.589247,Act. moderada,Act. baja,16.799999,16.700001,0.0,48.515068,3.133333,15.866667,1.0,7.0,Act. moderada,100.000
2,541,2.0,1,2012-03-15,Mujer,0.0,0.0,0,0,0,0,0,0,0,0,0,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,25.0,0.0,3.0,13.0,0.60,0.0,20.0,8.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.504439,2.369502,Remision,Act. baja,7.700000,7.100000,0.0,60.082191,8.666667,22.366667,1.0,3.5,Act. leve,1528.125
3,542,1.0,0,2012-06-21,Mujer,0.0,0.0,0,0,0,0,0,0,0,0,0,5.0,15.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,13.0,0.10,2.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.879465,1.293533,Remision,Remision,0.700000,0.600000,0.0,62.964382,6.533333,19.500000,0.0,0.0,Remision,1642.500
4,545,0.0,1,2012-03-01,Mujer,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,18.0,0.0,2.0,56.0,0.20,5.0,20.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.255726,1.793480,Act. moderada,Remision,4.300000,4.100000,0.0,69.490410,10.866667,23.733334,1.0,2.5,Act. leve,75.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,836,1.0,1,2019-05-15,Mujer,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,5.0,3.0,0.0,25.0,0.23,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.853162,2.989761,Act. moderada,Act. baja,8.230000,8.000000,NaN,63.967125,0.400000,13.233334,1.0,5.0,Act. leve,0.000
119,839,2.0,1,2018-06-15,Hombre,0.0,0.0,0,0,0,0,0,0,0,0,0,2.5,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,75.0,12.0,7.0,79.0,1.26,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.590000,NaN,Act. alta,NaN,33.259998,NaN,NaN,62.526028,12.066667,NaN,1.0,11.5,Act. alta,NaN
120,842,1.0,1,2019-05-15,Mujer,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.06,0.0,0.0,0.0,0.0,1250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.485203,1.129201,Remision,Remision,0.360000,0.300000,NaN,31.545206,1.233333,13.233334,1.0,0.0,Remision,0.000
121,843,0.0,1,2019-05-15,Mujer,0.0,15.0,0,0,0,0,0,0,0,0,0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.980824,1.566667,NaN,1.0,NaN,NaN,0.000


In [72]:
Madrid_additional

,codigo2,fechaini,vgepac_v1,vgemed_v1,nad_v1,nat_v1,vsg_v1,pcr_v1,das28_v1,das28pcr_v1,das28cat_v1,das28pcrcat_v1,sdai_v1,cdai_v1,vgepac_v2,vgemed_v2,nad_v2,nat_v2,vsg_v2,pcr_v2,das28_v2,das28pcr_v2,das28cat_v2,das28pcrcat_v2,sdai_v2,cdai_v2,vgepac_v3,vgemed_v3,nad_v3,nat_v3,vsg_v3,pcr_v3,das28_v3,das28pcr_v3,das28cat_v3,das28pcrcat_v3,sdai_v3,cdai_v3,peso,talla,bmi,edadinienf,tevol_v1,tevol_v2,tevol_v3,fecha1fme,hupi_v1,hupicat_v1,hupi_v2,hupicat_v2,hupi_v3,hupicat_v3,CDAI_S,SDAI_S
0,536,2012-08-15,47,25,1,2,46.0,0.60,4.294029,3.274508,Act. moderada,Act. baja,10.800000,10.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.00,-9.0,12.0,14.0,0.0,0.0,29.0,0.30,2.525107,1.627066,Remision,Remision,2.900000,2.600000,92.5,1.610,35.685352,65.180824,2.266667,NaN,14.400000,2012-10-23,6.0,Act. moderada,NaN,NaN,1.0,Remision,2,2
1,539,2012-08-20,27,22,2,2,19.0,0.10,3.627047,2.775472,Act. moderada,Act. baja,9.000000,8.900000,81.0,54.0,9.0,4.0,15.0,0.10,5.269635,4.583533,Act. alta,Act. moderada,26.60,26.5,52.0,45.0,6.0,1.0,21.0,0.10,4.510880,3.589247,Act. moderada,Act. baja,16.799999,16.700001,60.0,1.720,20.281233,48.515068,3.133333,9.433333,15.866667,2012-11-24,3.5,Act. leve,8.0,Act. moderada,7.0,Act. moderada,1,1
2,541,2012-03-15,45,58,2,9,50.0,4.00,5.000376,4.558846,Act. moderada,Act. moderada,25.299999,21.299999,29.0,35.0,0.0,3.0,20.0,0.30,2.987987,2.350040,Act. baja,Act. baja,9.70,9.4,16.0,25.0,0.0,3.0,13.0,0.60,2.504439,2.369502,Remision,Act. baja,7.700000,7.100000,55.0,1.470,25.452358,60.082191,8.666667,13.900000,22.366667,2012-12-05,9.0,Act. moderada,3.5,Act. leve,3.5,Act. leve,1,1
3,542,2012-06-21,55,75,10,10,17.0,0.20,5.409563,4.781814,Act. alta,Act. moderada,33.200001,33.000000,12.0,14.0,0.0,1.0,14.0,0.00,2.295340,1.408000,Remision,Remision,3.60,3.6,6.0,0.0,0.0,0.0,13.0,0.10,1.879465,1.293533,Remision,Remision,0.700000,0.600000,57.0,1.640,21.192743,62.964382,6.533333,14.233334,19.500000,2013-01-08,9.5,Act. alta,1.0,Remision,0.0,Remision,1,1
4,545,2012-03-01,65,49,7,9,40.0,1.30,5.813837,5.141681,Act. alta,Act. alta,28.700001,27.400000,8.0,10.0,0.0,1.0,37.0,0.10,2.919642,1.601533,Act. baja,Remision,2.90,2.8,3.0,18.0,0.0,2.0,56.0,0.20,3.255726,1.793480,Act. moderada,Remision,4.300000,4.100000,55.5,1.480,25.337837,69.490410,10.866667,16.933332,23.733334,2013-01-22,11.5,Act. alta,2.5,Act. leve,2.5,Act. leve,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,836,2019-05-15,66,25,10,1,31.0,0.50,5.378666,4.579909,Act. alta,Act. moderada,20.600000,20.100000,44.0,13.0,4.0,0.0,46.0,0.27,4.416049,3.167000,Act. moderada,Act. baja,9.97,9.7,45.0,5.0,3.0,0.0,25.0,0.23,3.853162,2.989761,Act. moderada,Act. baja,8.230000,8.000000,63.0,1.510,27.630367,63.967125,0.400000,6.233333,13.233334,2019-05-28,8.5,Act. moderada,5.5,Act. moderada,5.0,Act. leve,1,1
119,839,2018-06-15,55,37,9,4,58.0,0.37,5.852310,4.527122,Act. alta,Act. moderada,22.570000,22.200001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.00,-9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,1.665,23.807590,62.526028,12.066667,NaN,NaN,2019-06-14,9.5,Act. alta,NaN,NaN,NaN,NaN,2,2
120,842,2019-05-15,91,65,14,9,16.0,0.76,6.150140,5.943963,Act. alta,Act. alta,39.360001,38.599998,2.0,0.0,0.0,0.0,6.0,0.06,1.282232,1.157201,Remision,Remision,0.26,0.2,0.0,3.0,0.0,0.0,2.0,0.06,0.485203,1.129201,Remision,Remision,0.360000,0.300000,58.0,1.560,23.833004,31.545206,1.233333,6.700000,13.233334,2019-06-24,10.0,Act. alta,0.0,Remision,0.0,Remision,2,2
121,843,2019-05-15,90,65,0,10,35.0,0.71,4.634181,3.858509,Act. moderada,Act. moderada,26.209999,25.500000,10.0,5.0,0.0,1.0,10.0,0.56,2.031810,2.059345,Remision,Remision,3.06,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.5,1.560,26.093031,38.980824,1.566667,8.566667,NaN,2019-07-02,8.0,Act. moderada,1.5,Remision,NaN,NaN,2,2


In [18]:
Madrid_fam = pd.read_csv('./Pipelinerun_withPC/Madrid/mergedChr/madrid_merged.fam',sep='\s',header=None)
Madrid_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,1018_ARC796,1018_ARC796,0,0,0,-9
1,1020_ARC552,1020_ARC552,0,0,0,1
2,1042_HCSC06,1042_HCSC06,0,0,0,-9
3,1044_ARC602,1044_ARC602,0,0,0,1
4,1068_ARC646,1068_ARC646,0,0,0,2
...,...,...,...,...,...,...
136,922_ARC560,922_ARC560,0,0,0,1
137,944_HCSC12,944_HCSC12,0,0,0,-9
138,946_ARC611,946_ARC611,0,0,0,1
139,970_ARC666,970_ARC666,0,0,0,2


In [19]:
#V2 6month
Madrid_additional['sdai_v2'] = Madrid_additional['sdai_v2'].fillna(-9)
Madrid_additional['cdai_v2'] = Madrid_additional['cdai_v2'].fillna(-9)

In [20]:
Madrid_additional['CDAI_S'] = Madrid_additional['cdai_v2'].map(cDAI_S)
Madrid_additional['SDAI_S'] = Madrid_additional['sdai_v2'].map(sDAI_S)

In [22]:
Madrid_fam['id'] = Madrid_fam[0].str.replace('^.*_',"").str.replace("^.*C","").str.lstrip("0").astype(int)
Madrid_select = pd.merge(Madrid_additional,Madrid_fam,how='right',left_on='codigo2',right_on='id')
Madrid_select['CDAI_S'] = Madrid_select['CDAI_S'].fillna(-9)
Madrid_select['SDAI_S'] = Madrid_select['SDAI_S'].fillna(-9)
Madrid_select

/tmp/ipykernel_1807674/965280630.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Madrid_fam['id'] = Madrid_fam[0].str.replace('^.*_',"").str.replace("^.*C","").str.lstrip("0").astype(int)


,codigo2,fechaini,vgepac_v1,vgemed_v1,nad_v1,nat_v1,vsg_v1,pcr_v1,das28_v1,das28pcr_v1,das28cat_v1,das28pcrcat_v1,sdai_v1,cdai_v1,vgepac_v2,vgemed_v2,nad_v2,nat_v2,vsg_v2,pcr_v2,das28_v2,das28pcr_v2,das28cat_v2,das28pcrcat_v2,sdai_v2,cdai_v2,vgepac_v3,vgemed_v3,nad_v3,nat_v3,vsg_v3,pcr_v3,das28_v3,das28pcr_v3,das28cat_v3,das28pcrcat_v3,sdai_v3,cdai_v3,peso,talla,bmi,edadinienf,tevol_v1,tevol_v2,tevol_v3,fecha1fme,hupi_v1,hupicat_v1,hupi_v2,hupicat_v2,hupi_v3,hupicat_v3,CDAI_S,SDAI_S,0,1,2,3,4,5,id
0,796.0,2018-07-01,78.0,55.0,5.0,6.0,50.0,2.9,5.768471,5.214486,Act. alta,Act. alta,27.200001,24.299999,0.0,15.0,2.0,1.0,3.0,0.15,1.840988,2.361824,Remision,Act. baja,4.650000,4.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,1.55,26.638918,78.717812,0.733333,6.566667,NaN,NaT,11.0,Act. alta,2.0,Remision,NaN,NaN,1.0,1.0,1018_ARC796,1018_ARC796,0,0,0,-9,796
1,552.0,2012-02-01,76.0,52.0,6.0,8.0,82.0,4.3,6.312377,5.549982,Act. alta,Act. alta,31.100000,26.799999,48.0,15.0,6.0,1.0,25.0,0.30,4.576927,3.782780,Act. moderada,Act. baja,13.600000,13.300000,56.0,25.0,1.0,1.0,37.0,0.30,4.151642,3.083066,Act. moderada,Act. baja,10.400000,10.100000,60.0,1.52,25.969528,77.271233,12.866667,18.366667,24.900000,2013-02-22,11.0,Act. alta,6.0,Act. moderada,6.5,Act. moderada,1.0,1.0,1020_ARC552,1020_ARC552,0,0,0,1,552
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,-9.0,-9.0,1042_HCSC06,1042_HCSC06,0,0,0,-9,6
3,602.0,2014-02-01,85.0,49.0,9.0,6.0,54.0,0.6,6.348146,5.216385,Act. alta,Act. alta,29.000000,28.400000,50.0,14.0,0.0,0.0,23.0,0.20,2.894846,2.055501,Act. baja,Remision,6.600000,6.400000,55.0,10.0,0.0,1.0,6.0,0.20,2.304232,2.405500,Remision,Act. baja,7.700000,7.500000,66.0,1.52,28.566483,73.539726,2.866667,8.233334,15.000000,2014-04-29,11.0,Act. alta,3.0,Act. leve,4.0,Act. leve,1.0,1.0,1044_ARC602,1044_ARC602,0,0,0,1,602
4,646.0,2015-01-15,0.0,25.0,0.0,1.0,14.0,0.1,2.127340,1.489533,Remision,Remision,3.600000,3.500000,3.0,2.0,0.0,0.0,11.0,0.10,1.720527,1.251533,Remision,Remision,0.600000,0.500000,0.0,0.0,0.0,0.0,22.0,0.30,2.163730,1.459066,Remision,Remision,0.300000,0.000000,57.5,1.60,22.460938,63.191780,6.433333,12.500000,18.100000,2015-07-29,1.0,Remision,0.0,Remision,1.0,Remision,2.0,2.0,1068_ARC646,1068_ARC646,0,0,0,2,646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,560.0,2013-03-15,39.0,25.0,6.0,5.0,7.0,2.9,3.905950,4.728244,Act. moderada,Act. moderada,20.299999,17.400000,20.0,9.0,4.0,0.0,13.0,1.00,3.195465,3.223242,Act. baja,Act. baja,7.900000,6.900000,32.0,15.0,2.0,0.0,12.0,1.50,2.979394,3.198092,Act. baja,Act. baja,8.200000,6.700000,70.0,1.72,23.661438,73.008217,4.066667,8.733334,15.266666,2014-03-15,9.5,Act. alta,5.0,Act. leve,5.5,Act. moderada,1.0,1.0,922_ARC560,922_ARC560,0,0,0,1,560
137,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,-9.0,-9.0,944_HCSC12,944_HCSC12,0,0,0,-9,12
138,611.0,2014-07-01,75.0,49.0,12.0,8.0,NaN,3.0,NaN,5.978092,NaN,Act. alta,35.400002,32.400002,50.0,12.0,3.0,0.0,13.0,0.50,3.465413,3.274982,Act. moderada,Act. baja,9.700000,9.200000,25.0,3.0,0.0,0.0,4.0,0.10,1.320406,1.559533,Remision,Remision,2.900000,2.800000,72.0,1.52,31.163435,80.093147,1.733333,6.733333,14.566667,2014-08-23,12.0,Act. alta,4.0,Act. leve,1.0,Remision,1.0,1.0,946_ARC611,946_ARC611,0,0,0,1,611
139,666.0,2015-08-15,15.0,30.0,1.0,4.0,20.0,0.2,3.427013,2.685500,Act. moderada,Act. baja,9.700000,9.500000,19.0,5.0,0.0,0.0,28.0,0.10,2.598543,1.475533,Remision,Remision,2.500000,2.400000,30.0,15.0,0.0,1.0,33.0,0.14,3.147555,1.

In [23]:
Madrid_fam_cDAI = Madrid_select.loc[:,[0,1,2,3,4,'CDAI_S']]
Madrid_fam_sDAI = Madrid_select.loc[:,[0,1,2,3,4,'SDAI_S']]

In [49]:
Madrid_fam_sDAI['SDAI_S'] = Madrid_fam_sDAI['SDAI_S'].astype(int)
Madrid_fam_cDAI['CDAI_S'] = Madrid_fam_cDAI['CDAI_S'].astype(int)

In [51]:
Madrid_fam_cDAI

,0,1,2,3,4,CDAI_S
0,1018_ARC796,1018_ARC796,0,0,0,1
1,1020_ARC552,1020_ARC552,0,0,0,1
2,1042_HCSC06,1042_HCSC06,0,0,0,-9
3,1044_ARC602,1044_ARC602,0,0,0,1
4,1068_ARC646,1068_ARC646,0,0,0,2
...,...,...,...,...,...,...
136,922_ARC560,922_ARC560,0,0,0,1
137,944_HCSC12,944_HCSC12,0,0,0,-9
138,946_ARC611,946_ARC611,0,0,0,1
139,970_ARC666,970_ARC666,0,0,0,2


In [50]:
Madrid_fam_sDAI.to_csv("/exports/reum/CKe/Pipelinerun_withPC/TwoCompDAS/Madrid_merged_sDAI.fam",sep='\t',index=False,header=False)
Madrid_fam_cDAI.to_csv("/exports/reum/CKe/Pipelinerun_withPC/TwoCompDAS/Madrid_merged_cDAI.fam",sep='\t',index=False,header=False)

## TACERA

In [11]:
Tacera = openfile("/exports/reum/CKe/Tacera/RA-MAP_data_clinical.csv")
keyfile = openfile("/exports/reum/CKe/Tacera/OpenPseudonymised_RA-MAP DNA_IDs deidentified to date 02Dec16_with_HLA.xlsx")
Tacera_fam = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Tacera/Final_imputed/TACERA_HRCv1_1_FINAL.fam",sep='\s',header=None)

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [115]:
keyfile

,Digest,sample_id,Study,notes,sample_id.1,HLA_A_1,HLA_B_1,HLA_C_1,HLA_DPA1_1,HLA_DPB1_1,HLA_DRB1_1,HLA_DQA1_1,HLA_DQB1_1,HLA_A_2,HLA_B_2,HLA_C_2,HLA_DPA1_2,HLA_DPB1_2,HLA_DRB1_2,HLA_DQA1_2,HLA_DQB1_2
0,D3A02019792FDA6C8C6B9CA21512801610FB7176BAE732...,D-21781-1,RAMAP-tacera,exclude: duplicate subject IDs but genotypes d...,D-21781-1,3201,5501,303,103,401,401,301,302,301,1518,704,103,201,1101,501,301
1,D3A02019792FDA6C8C6B9CA21512801610FB7176BAE732...,D-21863-1,RAMAP-tacera,exclude: duplicate subject IDs but genotypes d...,D-21863-1,301,5701,602,201,1301,701,201,303,201,702,702,103,402,1103,501,301
2,NaN,D-21919-1,RAMAP-tacera,exclude: subjects unknown,D-21919-1,101,5701,602,201,1001,1501,102,602,201,1501,303,103,401,401,301,302
3,NaN,D-21955-1,RAMAP-tacera,exclude: subjects unknown,D-21955-1,2301,4403,401,103,201,701,201,202,301,1501,102,201,1001,101,101,501
4,NaN,D-22960-1,RAMAP-tacera,exclude: subjects unknown,D-22960-1,1101,3501,401,201,101,101,101,501,101,702,702,103,401,1101,501,301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,F77FCE78B9C12C4011E3C22AC81BD7EA93519F47769081...,D-23010-1,Vaccine,NaN,D-23010-1,201,4402,704,103,401,101,101,501,301,702,702,103,601,1501,102,602
291,E09DBC4609157185B2C8110022C52378534EAEFA20BA83...,D-33052-1,Vaccine,NaN,D-33052-1,6801,702,702,103,401,1501,102,602,1101,5601,102,201,1101,701,201,202
292,335879869F16C038AE5E619D36C979021B7D761AC80B84...,Z-21686-1,RAMAP-tacera,NaN,Z-21686-1,1101,4001,304,103,201,1501,102,502,2402,1301,304,103,401,1501,102,601
293,BF9D48FEB435B3F585DDDBF5D966C024E2027C62F82C48...,Z-21746-1,RAMAP-tacera,NaN,Z-21746-1,301,3501,401,103,401,401,301,301,1101,5101,1502,103,601,404,301,302


In [112]:
Tacera

,Subject ID,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA Unit\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA Value\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\ACPA\ACPA positive\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\RF Unit\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\RF Value\,\RAMAP\RAMAP\Clinical Data\.ACPA/RF Eligibility\Rheumatoid Factor\Rheum Factor positive\,\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA Unit SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA Value SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\ACPA\ACPA positive SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\RF Unit SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\RF Value SCR\,\RAMAP\RAMAP\Clinical Data\.Screening\Rheumatoid Factor\Rheum Factor positive SCR\,\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA Unit 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA Value 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\ACPA\ACPA positive 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Basophils 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Date of FBC 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Eosinophils 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\GL 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Gdl 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\HB 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Lymphocytes 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Monocytes 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\Neutrophils 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\PLT 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Blood count\WBC 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\CRP Value (mg/L) 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\ESR Value 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Evaluator global 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Fatique 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Outside 2 week window 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Pain score 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Patient global 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Total 66/68 Swollen Joints 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Total 66/68 Tender Joints 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Total Swollen 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Total Tender 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\Components\Use of sign '<' 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\DAS-28 CRP 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\DAS28 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\DAS\DAS28 relative 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\HAQ\Metrologist's score 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Rheumatoid Factor\RF Unit 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Rheumatoid Factor\RF Value 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\Rheumatoid Factor\Rheum Factor positive 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\SDAI\SDAI <= 3.3 00BL\,\RAMAP\RAMAP\Clinical Data\00 Baseline\SDAI\SDAI score 00BL\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\CRP Value (mg/L) 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\ESR Value 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Evaluator global 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Fatique 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Outside 2 week window 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Pain score 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Patient global 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Total 66/68 Swollen Joints 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Total 66/68 Tender Joints 03M\,\RAMAP\RAMAP\Clinical Data\03 Months\DAS\Components\Total Swollen 03M\,

In [12]:
Tacera['SDAI_S'] = Tacera.iloc[:,93].fillna(-9)
Tacera['SDAI_S'] = Tacera['SDAI_S'].map({'Yes':2,'No':1,-9:0})

In [13]:
Tacera_fam

,0,1,2,3,4,5
0,D-12945-1_D-12945-1,D-12945-1_D-12945-1,0,0,0,-9
1,D-21657-1_D-21657-1,D-21657-1_D-21657-1,0,0,1,1
2,D-21658-1_D-21658-1,D-21658-1_D-21658-1,0,0,2,2
3,D-21659-1_D-21659-1,D-21659-1_D-21659-1,0,0,2,2
4,D-21660-1_D-21660-1,D-21660-1_D-21660-1,0,0,2,1
...,...,...,...,...,...,...
290,D-23461-1_D-23461-1,D-23461-1_D-23461-1,0,0,0,-9
291,D-33052-1_D-33052-1,D-33052-1_D-33052-1,0,0,0,-9
292,Z-21686-1_Z-21686-1,Z-21686-1_Z-21686-1,0,0,2,1
293,Z-21746-1_Z-21746-1,Z-21746-1_Z-21746-1,0,0,2,1


In [14]:
Tacera = Tacera.merge(keyfile,how='left',left_on="Subject ID",right_on='Digest')

In [15]:
Tacera_fam['id'] = Tacera_fam[0].str.split("_",expand=True)[0]
Tacera_fam_sdai = pd.merge(Tacera_fam,Tacera,how='left',left_on='id',right_on='sample_id').loc[:,[0,1,2,3,4,'SDAI_S']]
Tacera_fam_sdai['SDAI_S'] = Tacera_fam_sdai['SDAI_S'].fillna(-9)
Tacera_fam_sdai

,0,1,2,3,4,SDAI_S
0,D-12945-1_D-12945-1,D-12945-1_D-12945-1,0,0,0,0.0
1,D-21657-1_D-21657-1,D-21657-1_D-21657-1,0,0,1,1.0
2,D-21658-1_D-21658-1,D-21658-1_D-21658-1,0,0,2,2.0
3,D-21659-1_D-21659-1,D-21659-1_D-21659-1,0,0,2,2.0
4,D-21660-1_D-21660-1,D-21660-1_D-21660-1,0,0,2,1.0
...,...,...,...,...,...,...
290,D-23461-1_D-23461-1,D-23461-1_D-23461-1,0,0,0,-9.0
291,D-33052-1_D-33052-1,D-33052-1_D-33052-1,0,0,0,0.0
292,Z-21686-1_Z-21686-1,Z-21686-1_Z-21686-1,0,0,2,1.0
293,Z-21746-1_Z-21746-1,Z-21746-1_Z-21746-1,0,0,2,1.0


In [52]:
Tacera_fam_sdai['SDAI_S'] = Tacera_fam_sdai['SDAI_S'].astype(int)

In [53]:
Tacera_fam_sdai.to_csv("/exports/reum/CKe/Pipelinerun_withPC/TwoCompDAS/Tacera_merged_sDAI.fam",sep='\t',index=False,header=False)

## SERA

In [123]:
SERA_fam = pd.read_csv('./Pipelinerun_withPC/SERA/mergedChr/sera_merged.fam',sep='\s',header=None)
SERA_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5
0,1_215284670,1_215284670,0,0,0,-9
1,100_215285060,100_215285060,0,0,0,1
2,1005_215284880,1005_215284880,0,0,0,2
3,1006_215293700,1006_215293700,0,0,0,-9
4,1007_215287480,1007_215287480,0,0,0,-9
...,...,...,...,...,...,...
663,984_215294600,984_215294600,0,0,0,2
664,985_215299420,985_215299420,0,0,0,2
665,986_215294170,986_215294170,0,0,0,2
666,987_215292670,987_215292670,0,0,0,-9


In [145]:
webjointfindings = openfile("/exports/reum/knevel_lab/SERA/Data/webJointFindings.CSV")
webjointfindings

,JFSNo,JFVisitID,ttlJntSwnTndr,ttlLrgJntSwnTndr,ttlSmllJntSwnTndr,ttlTndrJnt,ttlSwnJnt,sjc28,tjc28,upprLmbInvlvd,lwrLmbInvlvd,hasSymmetry,hasSmllJntHndFt
0,100,Baseline Visit,7,2,5,6,6,6,6,Yes,No,Yes,Yes
1,100,Month 12,2,1,1,1,1,1,1,Yes,Yes,No,Yes
2,100,Month 6,1,0,1,1,1,1,1,Yes,No,No,Yes
3,101,Baseline Visit,17,2,15,9,17,11,5,Yes,Yes,Yes,Yes
4,101,Month 12,0,0,0,0,0,0,0,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2696,1317,Month 6,17,1,16,17,15,15,17,Yes,No,Yes,Yes
2697,1318,Baseline Visit,22,2,20,22,20,20,22,Yes,No,Yes,Yes
2698,1318,Month 6,5,0,5,5,1,1,5,Yes,No,Yes,Yes
2699,1319,Baseline Visit,16,0,16,12,14,14,12,Yes,No,Yes,Yes


In [146]:
webjointfindings = webjointfindings[webjointfindings['JFVisitID']=='Month 6']
webjointfindings

,JFSNo,JFVisitID,ttlJntSwnTndr,ttlLrgJntSwnTndr,ttlSmllJntSwnTndr,ttlTndrJnt,ttlSwnJnt,sjc28,tjc28,upprLmbInvlvd,lwrLmbInvlvd,hasSymmetry,hasSmllJntHndFt
2,100,Month 6,1,0,1,1,1,1,1,Yes,No,No,Yes
5,101,Month 6,5,0,5,4,1,1,4,Yes,No,Yes,Yes
7,102,Month 6,2,0,2,0,2,2,0,Yes,No,No,No
10,103,Month 6,24,4,20,24,11,11,23,Yes,Yes,Yes,Yes
13,104,Month 6,5,0,5,5,0,0,5,Yes,No,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,1315,Month 6,6,0,6,4,3,3,4,Yes,No,Yes,Yes
2694,1316,Month 6,1,0,1,0,1,1,0,Yes,No,No,Yes
2696,1317,Month 6,17,1,16,17,15,15,17,Yes,No,Yes,Yes
2698,1318,Month 6,5,0,5,5,1,1,5,Yes,No,Yes,Yes


In [141]:
webDisease = openfile("/exports/reum/knevel_lab/SERA/Data/webDisease.CSV")
webDisease

,DiseaseSNo,DiseaseVisitID,ESR,DasVAS,CRP,GlobalVAS,PainVAS,RFMeasureType,RFFigure,RFTitrePosNeg,RFTitreUpper,RFTitreLower,CCPMeasureType,CCPFigure,CCPTitrePosNeg,CCPTitreUpper,CCPTiteLower,MrnStffnssID,IsESRDoneYN,IsCRPDoneYN,IsRFDoneYN,IsCCPDoneYN,RFFigureOperator,CCPFigureOperator,CRPFigureOperator,IsPainVASDoneYN,IsGlobalVASDoneYN,IsMrnStiffnessDoneYN,ESRNAYN,CRPNAYN,RFNAYN,CCPNAYN,PainVASNAYN,GlobalVASNAYN
0,100,Baseline Visit,24.0,17,11.0,60.0,11.0,Figure,197.0,NaN,NaN,NaN,Figure,141.0,NaN,NaN,NaN,> 30 mins,Yes,Yes,Yes,Yes,=,=,=,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
1,100,Month 12,5.0,1,0.6,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
2,100,Month 6,7.0,2,1.1,10.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
3,101,Baseline Visit,35.0,59,15.0,60.0,38.0,Figure,218.0,NaN,NaN,NaN,Figure,91.0,NaN,NaN,NaN,>= 60 mins,Yes,Yes,Yes,Yes,=,=,=,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
4,101,Month 12,13.0,2,1.8,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2696,1317,Month 6,NaN,65,NaN,25.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,Yes,No,No,No,No
2697,1318,Baseline Visit,NaN,95,4.0,50.0,100.0,NaN,NaN,NaN,NaN,NaN,Figure,0.6,NaN,NaN,NaN,>= 60 mins,No,Yes,No,Yes,=,=,<,Yes,Yes,Yes,No,No,No,No,No,No
2698,1318,Month 6,NaN,5,4.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,NaN,NaN,=,=,<,Yes,Yes,Yes,No,No,No,No,No,No
2699,1319,Baseline Visit,NaN,79,79.0,35.0,78.0,NaN,NaN,NaN,NaN,NaN,Figure,0.4,NaN,NaN,NaN,>= 60 mins,No,Yes,No,Yes,=,>,=,Yes,Yes,Yes,No,No,No,No,No,No


In [143]:
webDisease = webDisease[webDisease['DiseaseVisitID']=='Month 6']
webDisease

,DiseaseSNo,DiseaseVisitID,ESR,DasVAS,CRP,GlobalVAS,PainVAS,RFMeasureType,RFFigure,RFTitrePosNeg,RFTitreUpper,RFTitreLower,CCPMeasureType,CCPFigure,CCPTitrePosNeg,CCPTitreUpper,CCPTiteLower,MrnStffnssID,IsESRDoneYN,IsCRPDoneYN,IsRFDoneYN,IsCCPDoneYN,RFFigureOperator,CCPFigureOperator,CRPFigureOperator,IsPainVASDoneYN,IsGlobalVASDoneYN,IsMrnStiffnessDoneYN,ESRNAYN,CRPNAYN,RFNAYN,CCPNAYN,PainVASNAYN,GlobalVASNAYN
2,100,Month 6,7.0,2,1.1,10.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
5,101,Month 6,12.0,1,2.6,10.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
7,102,Month 6,5.0,0,0.7,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
10,103,Month 6,100.0,47,97.0,70.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
13,104,Month 6,7.0,86,1.4,50.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691,1315,Month 6,5.0,25,6.0,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
2694,1316,Month 6,NaN,20,0.7,10.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,No,No,No,No,No
2696,1317,Month 6,NaN,65,NaN,25.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,NaN,NaN,=,=,=,Yes,Yes,Yes,No,Yes,No,No,No,No
2698,1318,Month 6,NaN,5,4.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,NaN,NaN,=,=,<,Yes,Yes,Yes,No,No,No,No,No,No


In [151]:
SERA_merge = pd.merge(webjointfindings,webDisease,how='inner',left_on='JFSNo',right_on='DiseaseSNo').loc[:,['JFSNo','sjc28','CRP']]
SERA_merge['2cDAS'] = TwocompDAS(SERA_merge['sjc28'],SERA_merge['CRP'])
SERA_merge

,JFSNo,sjc28,CRP,2cDAS
0,100,1,1.1,1.445162
1,101,1,2.6,1.768560
2,102,2,0.7,1.732591
3,103,11,97.0,6.067605
4,104,0,1.4,0.525281
...,...,...,...,...
869,1315,3,6.0,2.899597
870,1316,1,0.7,1.318377
871,1317,15,NaN,NaN
872,1318,1,4.0,1.965663


In [152]:
def processdcID(row):
    row = str(row).replace(" ","").lstrip("0")
    return row
def processFAM(row):
    row = str(row).split("_")[1]
    return row

In [153]:
SERA_pheno = openfile("/exports/reum/knevel_lab/SERA/Data/SERA Final Leiden Manifest.xlsx",n=2)
SERA_pheno['ID'] = SERA_pheno['dcID'].map(processdcID)
SERA_pheno

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,StudyNo,Location,dcID,Barcode original,Plate,ID
0,434,"(1, 1)",02 1528731 0,NTBJ386054,74501.0,215287310
1,625,"(1, 10)",02 1528740 0,NTBJ391302,74501.0,215287400
2,517,"(1, 11)",02 1528741 0,NTBJ385994,74501.0,215287410
3,419,"(1, 12)",02 1528742 0,NTBJ386203,74501.0,215287420
4,533,"(1, 4)",02 1528734 0,NTBJ386522,74501.0,215287340
...,...,...,...,...,...,...
691,226,"(6, 9)",02 1529866 0,E0000000102,74514.0,215298660
692,326,"(6, 10)",02 1529867 0,E0000000459,74514.0,215298670
693,230,"(6, 11)",02 1529868 0,E0000000219,74514.0,215298680
694,286,"(6, 12)",02 1528752 0,E0000000704,74514.0,215287520


In [155]:
SERA_pheno_2cDAS = pd.merge(SERA_pheno,SERA_merge,how='inner',left_on='StudyNo',right_on='JFSNo')
SERA_pheno_2cDAS

,StudyNo,Location,dcID,Barcode original,Plate,ID,JFSNo,sjc28,CRP,2cDAS
0,625,"(1, 10)",02 1528740 0,NTBJ391302,74501.0,215287400,625,4,35.00,4.150111
1,517,"(1, 11)",02 1528741 0,NTBJ385994,74501.0,215287410,517,0,NaN,NaN
2,419,"(1, 12)",02 1528742 0,NTBJ386203,74501.0,215287420,419,0,8.00,1.318335
3,641,"(1, 5)",02 1528735 0,NTBJ386610,74501.0,215287350,641,0,4.00,0.965663
4,431,"(1, 6)",02 1528736 0,NTBJ385684,74501.0,215287360,431,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
466,1158,"(6, 4)",02 1529460 0,E0000002263,74514.0,215294600,1158,0,11.70,1.524961
467,228,"(6, 7)",02 1529864 0,E0000000131,74514.0,215298640,228,0,1.00,0.415888
468,954,"(6, 8)",02 1529865 0,E0000001949,74514.0,215298650,954,2,8.00,2.732548
469,286,"(6, 12)",02 1528752 0,E0000000704,74514.0,215287520,286,0,3.91,0.954764


In [154]:
SERA_fam['ID'] = SERA_fam[0].map(processFAM)
SERA_fam

,0,1,2,3,4,5,ID
0,1_215284670,1_215284670,0,0,0,-9,215284670
1,100_215285060,100_215285060,0,0,0,1,215285060
2,1005_215284880,1005_215284880,0,0,0,2,215284880
3,1006_215293700,1006_215293700,0,0,0,-9,215293700
4,1007_215287480,1007_215287480,0,0,0,-9,215287480
...,...,...,...,...,...,...,...
663,984_215294600,984_215294600,0,0,0,2,215294600
664,985_215299420,985_215299420,0,0,0,2,215299420
665,986_215294170,986_215294170,0,0,0,2,215294170
666,987_215292670,987_215292670,0,0,0,-9,215292670


In [160]:
SERA_2cDAS = pd.merge(SERA_pheno_2cDAS,SERA_fam,how='inner',left_on='ID',right_on='ID')
SERA_2cDAS['2cDAS'].fillna(0,inplace=True)
SERA_2cDAS

,StudyNo,Location,dcID,Barcode original,Plate,ID,JFSNo,sjc28,CRP,2cDAS,0,1,2,3,4,5
0,625,"(1, 10)",02 1528740 0,NTBJ391302,74501.0,215287400,625,4,35.00,4.150111,1156_215287400,1156_215287400,0,0,0,1
1,517,"(1, 11)",02 1528741 0,NTBJ385994,74501.0,215287410,517,0,NaN,0.000000,1967_215287410,1967_215287410,0,0,0,2
2,419,"(1, 12)",02 1528742 0,NTBJ386203,74501.0,215287420,419,0,8.00,1.318335,2014_215287420,2014_215287420,0,0,0,2
3,641,"(1, 5)",02 1528735 0,NTBJ386610,74501.0,215287350,641,0,4.00,0.965663,717_215287350,717_215287350,0,0,0,1
4,431,"(1, 6)",02 1528736 0,NTBJ385684,74501.0,215287360,431,2,NaN,0.000000,764_215287360,764_215287360,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,1158,"(6, 4)",02 1529460 0,E0000002263,74514.0,215294600,1158,0,11.70,1.524961,984_215294600,984_215294600,0,0,0,2
463,228,"(6, 7)",02 1529864 0,E0000000131,74514.0,215298640,228,0,1.00,0.415888,1815_215298640,1815_215298640,0,0,0,2
464,954,"(6, 8)",02 1529865 0,E0000001949,74514.0,215298650,954,2,8.00,2.732548,1768_215298650,1768_215298650,0,0,0,1
465,286,"(6, 12)",02 1528752 0,E0000000704,74514.0,215287520,286,0,3.91,0.954764,1155_215287520,1155_215287520,0,0,0,2


In [162]:
SERA_2cDAS.loc[:,[0,1,2,3,4,'2cDAS']].to_csv("/exports/reum/CKe/Pipelinerun_withPC/TwoCompDAS/sera_meged_2cDAS.fam",sep='\t',index=False,header=False)

### SERA2-GLASGOW

In [166]:
SERA2_fam = pd.read_csv("/exports/reum/CKe/Pipelinerun_withPC/Glasgow/mergedChr/Glasgow_merged.fam",sep='\s',header=None)
SERA2_fam['ID'] = SERA2_fam[0].map(processFAM)
SERA2_fam

/home/cke/.conda/envs/CKe/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3,4,5,ID
0,3_215289020,3_215289020,0,0,1,1,215289020
1,4_215285720,4_215285720,0,0,1,2,215285720
2,5_215287570,5_215287570,0,0,0,-9,215287570
3,6_215286070,6_215286070,0,0,2,2,215286070
4,10_215286430,10_215286430,0,0,1,2,215286430
...,...,...,...,...,...,...,...
339,568_215288500,568_215288500,0,0,2,1,215288500
340,569_215291730,569_215291730,0,0,2,1,215291730
341,572_215286600,572_215286600,0,0,2,1,215286600
342,574_215285670,574_215285670,0,0,2,2,215285670


In [167]:
SERA2_pheno = openfile("/exports/reum/knevel_lab/SERA/Data/Leiden Manifest 14 December 20.xlsx")
SERA2_pheno['ID'] = SERA2_pheno['dcID'].map(processdcID)
SERA2_pheno

,Box Number,StudyNo,dcID,Barcode original,Updated location,Scanned barcode,ID
0,Box 1 SERA Decode DNA,206,02 1528607 0,NTBI991424,A1,NaN,215286070
1,Box 1 SERA Decode DNA,179,02 1528608 0,NUFY858106,A2,NaN,215286080
2,Box 1 SERA Decode DNA,154,02 1528609 0,NUFY853350,A3,NaN,215286090
3,Box 1 SERA Decode DNA,139,02 1528610 0,NUFY851945,A4,NaN,215286100
4,Box 1 SERA Decode DNA,120,02 1528611 0,NUFY850894,A5,NaN,215286110
...,...,...,...,...,...,...,...
338,74505,453,02 1529086 0,NTBI997002,"(7, 2)",NaN,215290860
339,74505,403,02 1529087 0,NTBI981782,"(7, 3)",NaN,215290870
340,74505,457,02 1529088 0,NTBI997491,"(7, 4)",NaN,215290880
341,74505,327,02 1529089 0,NTBI989137,"(7, 5)",NaN,215290890
